Predict the closest block that will get the driver the highest tips based on the time of the day and day of the week.

In [10]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
data_2012_01=pd.read_csv("/Users/RAFAELLICHEN/Downloads/column_deleted_data_2012/yellow_tripdata_2012-01.csv")
data_2012_01=data_2012_01.drop("Unnamed: 0",1)
data_2012_01=data_2012_01.drop("passenger_count",1)
data_2012_01=data_2012_01.drop("dropoff_datetime",1)
data_2012_01=data_2012_01.drop("trip_distance",1)
data_2012_01=data_2012_01.drop("dropoff_longitude",1)
data_2012_01=data_2012_01.drop("dropoff_latitude",1)
data_2012_01=data_2012_01.drop("total_amount",1)
data_2012_01.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount
0,2012-01-10 23:55:50,-73.994693,40.725031,1.00
1,2012-01-11 19:18:25,-73.987955,40.752947,0.00
2,2012-01-11 19:19:19,-73.783093,40.648550,10.06
3,2012-01-11 19:19:21,-73.967515,40.758454,1.00
4,2012-01-11 14:38:15,-74.011315,40.711449,0.00


In [3]:
jan_time=pd.to_datetime(pd.Series(data_2012_01["pickup_datetime"]))
jan_time_list=[]
for index in jan_time:
    jan_time_list.append(index.dayofweek)
data_2012_01["weekday"]=jan_time_list
data_2012_01.head()

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday
0,2012-01-10 23:55:50,-73.994693,40.725031,1.00,1
1,2012-01-11 19:18:25,-73.987955,40.752947,0.00,2
2,2012-01-11 19:19:19,-73.783093,40.648550,10.06,2
3,2012-01-11 19:19:21,-73.967515,40.758454,1.00,2
4,2012-01-11 14:38:15,-74.011315,40.711449,0.00,2


In [4]:
jan_2012_mon=pd.DataFrame(data_2012_01)
jan_2012_mon=jan_2012_mon.loc[jan_2012_mon["weekday"]==0]
jan_2012_mon=jan_2012_mon.reset_index(drop=True)
print(jan_2012_mon.shape)

(2098157, 5)


In [5]:
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude>40]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude<41]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude>-74.5]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude<-72.5]
jan_2012_mon.shape

(2048831, 5)

In [6]:
jan_2012_mon.head(1)

,pickup_datetime,pickup_longitude,pickup_latitude,tip_amount,weekday
0,2012-01-09 19:14:00,-73.993335,40.727717,0.0,0


In [7]:
in_range=jan_2012_mon.loc[jan_2012_mon["pickup_longitude"]>=-73.993335-0.018]
in_range=in_range.loc[in_range["pickup_longitude"]<=-73.993335+0.018]
in_range=in_range.loc[in_range["pickup_latitude"]<=40.727717+0.018]
in_range=in_range.loc[in_range["pickup_latitude"]>=40.727717-0.018]
in_range.shape

(560846, 5)

In [12]:
# in_range.to_csv("2012-01-0-id.csv")

In [13]:
import matplotlib.path as mplPath

def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones("/Users/RAFAELLICHEN/Documents/CSC 59970/nyc-yellow-taxi-prediction/datasets/block-groups-polygons.geojson")
    i=0
    for line in parts:
        if (line["pickup_longitude"] and line["pickup_latitude"]):
            pickup_location  = geom.Point(proj(float(line["pickup_longitude"]), float(line["pickup_latitude"])))
            try:
                block_id_list.append(findBlock(pickup_location, index, zones))
            except AttributeError:
                drop_list.append(i)
        i+=1

drop_list=[]
block_id_list=[]

In [ ]:
mapToZone(in_range.T.to_dict().values())

In [28]:
print((len(drop_list)+len(block_id_list))==in_range.shape[0])

False


In [ ]:
jan_2012_mon=jan_2012_mon.drop(jan_2012_mon.index[drop_list])
jan_2012_mon["block_id"]=block_id_list
jan_2012_mon=jan_2012_mon.reset_index(drop=True)
jan_2012_mon=jan_2012_mon.loc[jan_2012_mon["block_id"]!=-1]
jan_2012_mon.head()